In [1]:
"""Collect, process, and format data used for model fitting."""
import math
import datetime as dt
import pandas as pd
import numpy as np
import yfinance as yf
from scipy.interpolate import CubicSpline
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [2]:
# file_names
spot_exchange_file = "data/EUR_USD Historical Data.csv"
sofr_file = "data/sofr-historical.csv"
euro_short_rate_file = "data/ECB Data Portal_20241214201559.csv"
yield_file = 'data/zero_coupon_annual_yields_11-22-2024.csv'
sofr_forward_file = 'data/sofr-forwards.csv'
cap_file = 'data/cap_bloomberg_vcub.csv'
options_file = 'data/combined_options_chain.csv'

In [3]:
# estimate rf (euro short term rate)
def rf_time_series(file):
    esr = pd.read_csv(file)
    esr = esr.set_index('DATE')
    esr.rename(columns={'Euro short-term rate - Volume-weighted trimmed mean rate (EST.B.EU000A2X2A25.WT)': 'eu-short-term-rate'}, inplace=True)
    return esr[esr['eu-short-term-rate'] > 0]

def rf_estimate(esr):
    return esr['eu-short-term-rate'].mean()

rf_series = rf_time_series(euro_short_rate_file)
rf = rf_estimate(rf_series) / 100
rf

np.float64(0.031248110918544195)

In [4]:
# estimate a (speed of mean reversion)
def rd_time_series(file):
    us_sr = pd.read_csv(file)
    us_sr['Effective Date'] = pd.to_datetime(us_sr['Effective Date'])
    us_sr = us_sr.set_index('Effective Date')
    us_sr.rename(columns={'Rate (%)' : 'us-short-term-rate'}, inplace=True)
    return us_sr['us-short-term-rate'].dropna() / 100

def estimate_a(us_sr, k=365):
    now = us_sr.iloc[:-k]
    then = us_sr.iloc[k:]
    reg = LinearRegression().fit(now.to_frame(), then)
    return reg.coef_

"""
# This currently is INCREDIBLY sensitive to choice of k days of lag. 
# I do not know any ways to feasibly justify one k value over the other
# Professor Javaheri suggested "a" should be between 0.03 and 0.06. 
# Choosing k = 390 does achieve something in this range, but this is incredibly unmotivated
"""    
rd_series = rd_time_series(sofr_file)
a_regression =  estimate_a(rd_series, k=390)[0]
a_regression

np.float64(0.04304488614334254)

In [5]:
# estimate correlations with the equity
def equity_time_series(ticker='^STOXX50E'):
    stock = yf.Ticker(ticker).history(period='10y', interval='1d')['Close']
    stock = stock.rename('stock-close')
    stock.index = stock.index.date
    return stock

def exchange_time_series(file):
    eur_usd = pd.read_csv(file)
    eur_usd['Date'] = pd.to_datetime(eur_usd['Date'])
    eur_usd = eur_usd.set_index('Date')
    eur_usd.index = eur_usd.index.date
    eur_usd.rename(columns={'Price' : 'exchange-rate'}, inplace=True)
    return eur_usd['exchange-rate'] ** (-1)

def returns(ts):
    return (ts.div(ts.shift(1)) - 1).iloc[1:]

stock_series = equity_time_series()
stock_returns = returns(stock_series)

fx_series = exchange_time_series(spot_exchange_file)
fx_returns = returns(fx_series)

def fx_stock_correlation(s_ts, fx_ts):
    joint = fx_ts.to_frame().join(s_ts, how='inner')
    return joint['exchange-rate'].corr(joint['stock-close'])

rho_sx = fx_stock_correlation(stock_returns, fx_returns)
print(rho_sx)

rd_returns = returns(rd_series)
def rates_stock_correlation(s_ts, r_ts):
    joint = r_ts.to_frame().join(s_ts, how='inner')
    return joint['us-short-term-rate'].corr(joint['stock-close'])

rho_sr = rates_stock_correlation(stock_returns, rd_returns)
print(rho_sr)

0.05983603261473958
0.03683283353786299


In [6]:
# current forward prices
def build_forward_dataframe(file):
    df = pd.read_csv(file)
    df.rename(columns={'3-month Term SOFR' : '3-month', '1-month Term SOFR' : '1-month'}, inplace=True)
    return df

forward_df = build_forward_dataframe(sofr_forward_file)

In [7]:
def cap_dataframe(file):
    df = pd.read_csv(file)
    df['Expiry'] = pd.to_datetime(df['Expiry'])
    df['Years'] = df['Days'].cumsum() / 365
    df['Cap Vols'] = df['Cap Vols'] / 10000  # data given in bps
    return df[['Years', 'Cap Vols']]

cap_df = cap_dataframe(cap_file)
print(cap_df)

# for T >= 5, choose closest 
# NOTE: this must be checked/updated every time you change the deal terms
sigma_r = 0.008428
print(sigma_r)

      Years  Cap Vols
0  0.501370  0.007042
1  1.000000  0.007977
2  1.501370  0.008272
3  2.000000  0.008739
4  2.501370  0.008739
5  3.008219  0.008558
6  3.506849  0.008557
7  4.005479  0.008431
8  4.504110  0.008428
0.008428


In [8]:
# current options prices (Bloomberg)
def get_options_chain(file):
    df = pd.read_csv(file)
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values(by=['Date', 'Strike'], inplace=True)
    df.reset_index(inplace=True)
    df['Years'] = df['Date'].apply(lambda x: (x - pd.to_datetime('12/07/2024')).total_seconds() / 31536000)
    return df

options_df = get_options_chain(options_file)

# specific to 5 years
display(options_df[(options_df['Years'].between(4.9, 5.1)) & ((options_df['C-Volm'] > 0) | (options_df['P-Volm'] > 0))])

# choose which option to imply volatility from
# NOTE: this must be checked/updated every time you change the deal terms
sigma_f = 0.2237384
print(sigma_f)

,index,Strike,Date,C-Ticker,C-Bid,C-Ask,C-Last,C-IVM,C-Volm,P-Ticker,P-Bid,P-Ask,P-Last,P-IVM,P-Volm,Years
1939,1934,2500,2029-12-01,SX5E 12/29 C2500,0.0,0.0,2276.100098,23.449985,0,SX5E 12/29 P2500,0.0,0.0,94.300003,0.000000,25000,4.986301
1956,1953,4000,2029-12-01,SX5E 12/29 C4000,0.0,0.0,1158.800049,0.000000,0,SX5E 12/29 P4000,0.0,0.0,336.399994,28.299494,25000,4.986301
1965,1962,4500,2029-12-01,SX5E 12/29 C4500,0.0,0.0,860.400024,0.000000,0,SX5E 12/29 P4500,0.0,0.0,491.200012,22.373840,12500,4.986301
1982,1981,6000,2029-12-01,SX5E 12/29 C6000,0.0,0.0,296.399994,15.788446,1000,SX5E 12/29 P6000,0.0,0.0,1286.500000,0.000000,0,4.986301


0.2237384


In [9]:
# parameters pulled from website data (not scraped)
q = 0.00068  # as of 12/15/2025
sigma_x = 0.0772  # as of 12/15/2024

In [10]:
def empirical_yield_curve(filename):
    """Use a cubic spline model to build treasury spot rate curve."""
    sc = pd.read_csv(filename)
    # Currently, adding overnight rate as "1 day zero coupon bond"
    return CubicSpline(sc['Years'], sc['Fitted Zero Coupon Yield'] / 100)

# -d/dt(- t*y(0, t))
def inst_forward_curve(times, yc):
    return np.gradient(times * yc(times), times)

t = np.arange(0, 10.01, 0.01)
yc = empirical_yield_curve(yield_file)
fc = CubicSpline(t, inst_forward_curve(t, yc))

In [11]:
def g(a, sigma, T):
    """Helper function for Hull-White calibration."""
    return (sigma ** 2) / (2 * a ** 2) * (1 - np.exp(-a * T)) ** 2

def gprime(a, sigma, T):
    """Helper derivative for Hull-White calibration."""
    return sigma ** 2 / a ** 2 * (1 - np.exp(-a * T)) * (a * np.exp(-a * T))

def theta(fc, a, sigma, t):
    return fc(t, 1) + gprime(a, sigma, t) + a*(fc(t) + g(a, sigma, t))
    
    

In [12]:
param_summary = {
    "S0" : stock_series.iloc[0],
    "r0" : rd_series.iloc[0],
    "L0-1month" : forward_df['1-month'][0],  # must be set such that it agrees with the delta routine below
    "L0-3month" : forward_df['3-month'][0],
    "rf" : rf,
    "q"  : q,
    "rho_sx" : rho_sx,
    "sigma_x" : sigma_x,
    "sigma_f" : sigma_f,
    "a" : a_regression,
    "sigma_r": sigma_r,
    "theta": theta,
    "rho_sr" : rho_sr,
}
display(param_summary)

# if a user (or grader) would like to manually play with parameters
param_user_override = {
    "S0" : None,
    "r0" : None,
    "L0-1month" : None,
    "L0-3month" : None,
    "rf" : None,
    "q"  : None,
    "rho_sx" : None,
    "sigma_x" : None,
    "sigma_f" : None,
    "a" : None,
    "sigma_r": None,
    "theta": None,
    "rho_sr" : None,
}

{'S0': np.float64(3049.989990234375),
 'r0': np.float64(0.0462),
 'L0-1month': np.float64(0.044),
 'L0-3month': np.float64(0.0436),
 'rf': np.float64(0.031248110918544195),
 'q': 0.00068,
 'rho_sx': np.float64(0.05983603261473958),
 'sigma_x': 0.0772,
 'sigma_f': 0.2237384,
 'a': np.float64(0.04304488614334254),
 'sigma_r': 0.008428,
 'theta': <function __main__.theta(fc, a, sigma, t)>,
 'rho_sr': np.float64(0.03683283353786299)}

In [13]:
# Monte Carlo Shit
def correlated_normal(rho, N):
    un_corr = np.random.normal(size=(2, N))
    L = np.array([[1, 0], [rho, np.sqrt(1 - rho**2)]])
    return L @ un_corr

def stock_path(S0, mu, sigma, t_step, N, deviations):
    noise = sigma * np.sqrt(t_step) * deviations
    growth_rate = 1 + t_step * mu + noise
    growth_rate = np.insert(growth_rate, 0, 1)
    return S0 * np.cumprod(growth_rate)

def rate_path(r0, a, sigma, theta, t_step, N, deviations):
    """Return the rate over the next N time steps starting from r0."""
    noise = sigma * np.sqrt(t_step) * deviations
    rate_path = np.empty(N + 1)
    rate_path[0] = r0
    for j in range(0, N):
        rate_path[j + 1] = rate_path[j] + (theta(fc, a, sigma, t_step * j) - a * rate_path[j]) * t_step + sigma * noise[j - 1]

    return rate_path

In [14]:
# Main routine
def price_hybrid(params, notional=1000000, k=1, kprime=1, T=5, delta=0.25, t_step=0.01, n_sim=1000):
    N = int(np.ceil((T + delta) / t_step))
    total = 0
    for j in range(n_sim):
        deviations = correlated_normal(params["rho_sr"], N)
        stock_sim = stock_path(
            params['S0'], 
            params['rf'] - params['rho_sx'] * params['sigma_x'] * params['sigma_f'],
            params['sigma_f'],
            t_step,
            N,
            deviations[0]
        )
        short_rate_sim = rate_path(
            params['r0'],
            params['a'],
            params['sigma_r'],
            params['theta'],
            t_step,
            N,
            deviations[1]
        )

        # compute value of assets
        M = int(T / t_step)
        ST = stock_sim[M]
        bond = np.exp(np.sum(short_rate_sim[M+1:]) * t_step)
        discount = np.exp(-np.sum(short_rate_sim[:M+1] * t_step))
        LT = (1 - bond) / (delta * bond)

        # compute value of option
        total += discount * max(0, (ST / params["S0"] - k) * (kprime - LT / params["L0-3month"]))

    return notional * total / n_sim

In [17]:
# output a price!
price_hybrid(param_summary)

np.float64(492420.42274317314)